# Solving the 1-D Diffusion Equation (Implicit)

Let us now apply the implicit finite difference scheme to our 1-D heat diffusion problem. To recap, here is the problem once again:

## The Problem

We are interested in how the temperature $T$ changes with time $t$, for a given initial temperature $T_{0}[x,t_{0}]$ and boundary conditions; for example, a dike intrusion in the lithosphere, with

* a total length $L$ = 100 m,
* a dike width $W$ = 5 m,
* a thermal diffusivity $\kappa$ = 10⁻⁶ m²/s,
* a dike temperature $T_{dike}$ = 1200 °C, and
* a background temperature $T_{background}$ = 300 °C.

<img src="./Figures/Exercise02_1b.png" alt="drawing" width="350"/> <br>
**Fig. 1.** Sketch of the geological problem and the profile of the initial temperature condition.

How long would it take for the dike to cool to a given temperature?

To determine the cooling time, we need to solve the temperature diffusion equation (a parabolic partial differential equation (PDE)):

$\begin{equation}
\rho c_p \frac{\partial{T}}{\partial{t}} = \frac{\partial{}}{\partial{x}} \left( k\frac{\partial{T}}{\partial{x}} \right),
\end{equation}$

where $k$ is the thermal conductivity [W/m/K], $T$ is the temperature [K], and $\rho$ is the density [kg/m³]. By rearranging the equation (assuming the thermal parameters are constant!), we can express it using the thermal diffusivity $\kappa = \frac{k}{\rho c_p}$:

$\begin{equation}
\frac{\partial{T}}{\partial{t}} = \kappa \frac{\partial^2{T}}{\partial{x^2}}.
\end{equation}$


### Finite Difference Approximation

To solve the problem numerically using finite differences, we first need to construct a numerical grid (the discretization):

<img src="../Figures/Exercise02_2.png" alt="drawing" width="600"/> <br>
**Fig. 2.** Numerical 1-D grid for the discretization of the *PDE*.

To solve the diffusion equation numerically using the implicit finite difference scheme, we must rewrite the *PDE* as follows:

$\begin{equation}
\frac{T_{I^\textrm{C}}^{n+1}-T_{I^\textrm{C}}^{n}}{\Delta t} = \frac{T_{I^\textrm{E}}^{n+1} - 2T_{I^\textrm{C}}^{n+1} + T_{I^\textrm{W}}^{n+1}}{(\Delta x)^2}.
\end{equation}$

This equation can be rearranged such that we obtain a system of equations with as many equations as there are central grid points:

$\begin{equation}
-a T_{I^\textrm{W}}^{n+1} + \left(2a + b\right) T_{I^\textrm{C}}^{n+1} - a T_{I^\textrm{E}}^{n+1} = b T_{I^\textrm{C}}^{n},
\end{equation}$

with
$\begin{equation}
a=\frac{\kappa}{\Delta{x^2}}, \quad b = \frac{1}{\Delta{t}}.
\end{equation}$

That is, we obtain a tridiagonal (three-diagonal) system of equations, which can be described by a coefficient matrix $\mathbf{A}$, an unknown vector $T^{n+1}$, and a known vector $T^n$:

$\begin{equation}
\mathbf{K} \cdot \bm{x} = \bm{b}
\end{equation}$

where $\mathbf{K}$ is the coefficient matrix (with three non-zero diagonals), $\bm{x}$ is the unknown solution vector (i.e., the temperature at time step $n+1$), and $\bm{b}$ is the known right-hand side.

#### General Solution

A general approach to solving this system of equations is **defect correction**. The heat diffusion equation is reformulated by introducing a residual term $\bm{r}$, which quantifies the deviation from the exact solution and can be iteratively reduced through successive correction steps to improve accuracy. In implicit form, Equation (6) can be rewritten as:

$\begin{equation}
\mathbf{K} \cdot \bm{x} - \bm{b} = \bm{r},
\end{equation}$

where $\bm{r}$ is the residual (or defect). The coefficients of the matrix are the same as derived above (Equation (5)). Within the defect correction framework, the coefficient matrix for a 1D problem with constant thermal parameters can be assembled using the built-in function `AssembleMatrix1Dc!()`.

The correction term is given by:

$\begin{equation}
\delta \bm{T} = -\mathbf{K}^{-1} \bm{r}^k.
\end{equation}$

Thus, the updated solution becomes:

$\begin{equation}
\bm{T}^{k+1} = \bm{T}^k + \delta \bm{T},
\end{equation}$

where $\bm{T}^{k+1}$ is the updated temperature after one iteration step.

---

In `GeoModBox.jl`, the residual $\bm{r}$ is computed at the centroids, using the extended temperature field—including the ghost nodes of the current time step—as an initial guess:

$\begin{equation}
\frac{\partial{T_{\textrm{ext,I}}}}{\partial{t}} - \kappa \frac{\partial^2{T_{\textrm{ext,I}}}}{\partial{x^2}} - \frac{Q}{\rho_0 c_p} = r_{I},
\end{equation}$

and in discretized, implicit finite-difference form:

$\begin{equation}
\frac{T_{\textrm{ext},I^\textrm{C}}^{n+1} - T_{\textrm{ext},I^\textrm{C}}^{n}}{\Delta{t}} - \kappa \frac{T_{\textrm{ext},I^\textrm{W}}^{n+1} - 2 T_{\textrm{ext},I^\textrm{C}}^{n+1} + T_{\textrm{ext},I^\textrm{E}}^{n+1}}{\Delta{x^2}} - \frac{Q_{I^\textrm{C}}^n}{\rho_0 c_p}= r_{I},
\end{equation}$

where $I^\textrm{C}=2:(nc+1)$ denotes the index of the centroids in the extended temperature field and $I=1:nc$ denotes the equation number. Rearranging Equation (11) and substituting the coefficients using Equation (5) yields:

$\begin{equation}
-aT_{\textrm{ext},I^\textrm{W}}^{n+1}
+\left(2a+b\right)T_{\textrm{ext},I^\textrm{C}}^{n+1}
-aT_{\textrm{ext},I^\textrm{E}}^{n+1}
-bT_{\textrm{ext},I^\textrm{C}}^n
-\frac{Q_{I^\textrm{C}}^n}{\rho_0 c_p}=r_{I},
\end{equation}$

which corresponds to the matrix form of Equation (7), where $T_{\textrm{ext},I^\textrm{C}}^{n+1}$ is the unknown vector $x$, $-bT_{\textrm{ext},I^\textrm{C}}^n -\frac{Q_{I^\textrm{C}}^n}{\rho_0 c_p}$ is the known vector $b$, and $-a$ as well as $2a+b$ are the coefficients of the non-zero diagonals of the coefficient matrix.

The residual can be computed using the built-in function `ComputeResiduals1Dc!()`. With the residual vector $\bm{r}$ and the coefficient matrix $\bm{K}$, the correction term for the temperature can be computed via Equation (8).

#### Special Case – A Linear Problem

If the problem is linear and the exact solution is reached within a single iteration step, the system of equations reduces to Equation (6). In this case, the system can be solved directly via *left matrix division*:

$\begin{equation}
\bf{x} = \mathbf{K}^{-1} \bm{b}.
\end{equation}$

The coefficient matrix remains unchanged, even for the given boundary conditions. However, the right-hand side must be updated accordingly (setting $\bm{r}=0$ and adding the known parameters to the right-hand side of the equations).

Here, we first formulate the special solution before applying the general, built-in solver.

For more information on solving PDEs using an implicit finite difference discretization, including the corresponding conditions as well as advantages and disadvantages, please refer to the relevant [documentation](https://geosci-ffm.github.io/GeoModBox.jl/dev/man/DiffOneD).


Let us first load the necessary modules for the numerical solution of our problem. We will begin by programming the solvers ourselves, before using the predefined solvers in GeoModBox.jl. The predefined solver is located in the submodule GeoModBox.HeatEquation.OneD.

In [ ]:
using Plots, ExtendableSparse, LinearAlgebra, Printf
using ?

### Parameter Definitions

Let us first define the parameters for the problem (physical constants, numerical domain, and time parameters).

The parameter `alternative` describes different programming options:

1. Fully explicit implementation (special case)
2. Use of the built-in function (special case)
3. Use of the general solution using "*defect correction*"


In [ ]:
alternative =   1
# Physics --------------------------------------------------------------- #
L           =   ?       # Length [m]
Tdike       =   ?       # Dike Temperature [C]
Trock       =   ?       # Backgraound Temperature [C]
κ           =   ?       # Thermal Diffusivity [m2/s]
W           =   ?       # Dike width [m]
# ----------------------------------------------------------------------- #
# Numerical Parameter --------------------------------------------------- #
nc          =   ?               # Number of centroid
Δx          =   ?               # Grid spacing
xc          =   ?               # x-coordinates of the centroids
# Iterations --  defection correction method
niter       =   50  
ϵ           =   1.0e-15       
# ----------------------------------------------------------------------- #
# Time Parameter -------------------------------------------------------- #
day         =   3600.0*24.0     # Seconds per day
fac         =   5.0             # multiplication factor for dt
Δt          =   ?               # time step length
tmax        =   2.0*365.0*day 
nt          =   ceil(Int,tmax/Δt)
Time        =   0
# ----------------------------------------------------------------------- #

### Initial Conditions

To solve our problem, we still need to define the initial conditions. We assume 300 °C for the host rock and 1200 °C for the dike. That is, the initial temperature is defined by:

$\begin{equation}
T \left(x < \left( \frac{L}{2} - W \right), x > \left( \frac{L}{2} + W \right), t = 0 \right) = 300,
\end{equation}$
$\begin{equation}
T \left(x > \left( \frac{L}{2} - W \right), x < \left( \frac{L}{2} + W \right), t = 0 \right) = 1200.
\end{equation}$

Now, we first want to visualize the initial condition.

In [ ]:
# Initial Condition; Temperature profile -------------------------------- #
T       =   (
        T       =   zeros( ? ),         # Centroid temperature
        T0      =   zeros( ? ),         # Old temperature field
        T_ex    =   zeros( ? ),         # Including ghost nodes
        T_ex0   =   zeros( ? ),         # Old temperature field 
        R       =   zeros( ? ))         # Residual
∂2T =   (
        ∂x2     =   zeros( ? ),         # Second derivative of T in space
        ∂x20    =   zeros( ? ),         # Old field
)
T.T    .= Trock                         # Background T
@. T.T[abs(xc-L/2) <= W/2] =  Tdike     # Dike T
T.T_ex[2:end-1]         .=      T.T
T.T0                    .=      T.T
# ----------------------------------------------------------------------- #
# Plot initial condition ------------------------------------------------ #
p = plot( ? )
display(p)
# ----------------------------------------------------------------------- #

### Boundary Conditions

Since we use cell-centered grid points (centroids) for the temperature, no grid point lies directly on the boundaries (contrary to expectations, this is actually an advantage!). To prescribe the temperature boundary conditions, we therefore need to use additional *ghost nodes* (see Figure 2), i.e., we determine the temperature on the *ghost nodes* in order to solve the temperature partial differential equation at the **nearest interior** grid point using finite differences. For a constant temperature boundary condition at the boundaries (Dirichlet), we can determine the temperature at the *ghost nodes* by linear interpolation such that:

**West**
$\begin{equation}
T_{Ghost}^W = 2 T_{BC}^W + T_{1},
\end{equation}$
**East**
$\begin{equation}
T_{Ghost}^E  = 2 T_{BC}^E + T_{nc}.
\end{equation}$

For constant flux boundary conditions, the temperature at the boundaries is given by:

**West**
$\begin{equation}
T_{Ghost}^W = T_{1} - c^W \Delta x,
\end{equation}$
**East**
$\begin{equation}
T_{Ghost}^E = T_{1} + c^E \Delta x,
\end{equation}$

where $c^W = \frac{\partial{T}}{\partial{x}}$ and $c^E = \frac{\partial{T}}{\partial{x}}$ define the flux conditions at the respective boundary.

In the case of the general defect-correction solver, the boundary conditions are incorporated by computing the residual at the outer centroids using the temperatures at the ghost nodes. That is, apart from the modified coefficients for the corresponding centroids, no special changes are required.

For the special case, the coefficients and the right-hand side of the system of equations for the **first** and **last interior grid point** must be modified depending on the boundary conditions as follows:

#### **Dirichlet**

*West*
$\begin{equation}
\left(3 a + b \right) T_{1}^{n+1} - a T_{2}^{n+1} = b T_{1}^{n} + 2 a T_{BC}^W
\end{equation}$
*East*
$\begin{equation}
-a T_{nc-1}^{n+1} + \left(3 a + b \right) T_{nc}^{n+1} = b T_{nc}^{n} + 2 a T_{BC}^E
\end{equation}$

#### **Neumann**

*West*
$\begin{equation}
\left(a + b \right) T_{1}^{n+1} - a T_{2}^{n+1} = b T_{1}^n - a c^{W} \Delta{x}
\end{equation}$
*East*
$\begin{equation}
-a T_{nc-1}^{n+1} + \left(a + b \right) T_{nc}^{n+1}  = b T_{nc}^n - a c^{E} \Delta{x}
\end{equation}$

When initializing the boundary conditions, we make use of the tuple `BC`. The tuple defines the type of boundary condition (`type`) and the required value (`val`). In the solution, we use `if` statements and logical parameters to modify the coefficients of the coefficient matrix and the right-hand side of the system of equations.


Since we have a tridiagonal system of equations, the coefficients and the right-hand side for the equations of the first and last interior grid points must be modified depending on the boundary conditions as follows (derivation see lecture):

#### **Dirichlet**
*West*
$$
\left(3 a + b \right) T_{1}^{n+1} - a T_{2}^{n+1} = b T{1}^{n} + 2 a T_{BC,W} \tag{12}
$$
*East*
$$
- a T_{nc-1}^{n+1} + \left(3 a + b \right) T_{nc}^{n+1} = b T_{nc}^{n} + 2 a T_{BC,E} \tag{13}
$$

#### **Neumann**
*West*
$$
\left(a + b \right) T_{1}^{n+1} - a T_{2}^{n+1} = b T_{1}^n - a c_{W} \Delta{x} \tag{14}
$$
*East*
$$
- a T_{nc-1}^{n+1} + \left(a + b \right) T_{nc}^{n+1}  = b T_{nc}^n - a c_{E} \Delta{x} \tag{15}
$$

When initializing the boundary conditions, we make use of a small trick involving the `BC` tuple. The tuple defines the type of boundary condition (`type`) and the associated value (`val`). In the solution, we use `if` statements and logical parameters to modify the coefficients of the coefficient matrix and the right-hand side of the linear system.


In [ ]:
# Boundary Conditions --------------------------------------------------- #
BC   = (
    type = (W=:Dirichlet, E=:Dirichlet),
    # type = (W=:Neumann, E=:Neumann),
    val  = (W=300.0, E=300.0))
#end
# ----------------------------------------------------------------------- #

### System of Equations

Let us now initialize the coefficient matrix and the right-hand side:

In [ ]:
# Assemble Coefficient Matrix ------------------------------------------- #
# Definition of the Matrix (in sparse form)
ndof        =   length(T.T)
K           =   ExtendableSparseMatrix(ndof,ndof)
rhs         =   zeros(nc)
# ----------------------------------------------------------------------- #

### Visualization

To visualize the results as an animation in a GIF file, we first need to specify the location and the name of the file:

In [ ]:
# Animationssettings ---------------------------------------------------- #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("03_1D_implicit_",alternative)
save_fig    =   1
# ----------------------------------------------------------------------- #          

### Time Loop

Numerically, we can now solve the *PDE* in a time loop in different ways (Alternative I is sufficient; those who want to can also implement Alternatives II and III).

To implement the boundary conditions, we use the logical parameters `DirW, NeuW, DirE, NeuE` as well as the parameters `inE` and `inW`. The latter describe whether a coefficient for the off-diagonals (*West* or *East*) is required (`true`) or not (`false`).

The logical parameters are either 0 (false) or 1 (true) for the respective boundary, depending on the chosen boundary condition (Dirichlet or Neumann). Accordingly, we can use these parameters to modify the coefficients and the right-hand side, e.g., (main diagonal and right-hand side, Dirichlet, West; see Equation (20)):

$$
\left((2+\textrm{DirW}) \cdot a + b\right) \cdot T_1^{n+1} - a T_{2}^{n+1} = b T_{1}^{n} + \textrm{DirW} \cdot (2 a T_{BC,W}).
$$

In the loop over the grid, we can formulate a general expression that accounts for all boundary conditions at all boundary points.

In [ ]:
# Timestep loop --------------------------------------------------------- #
for n = 1:nt
    println("Time Step: ",n,", Time: $(round(Time/day, digits=1)) [d]")
    if alternative == 1
        a   =   ?
        b   =   ? 

        @. rhs     =   ?

        # Alternative I
        for i = 1:nc  
            # Equation number
            ii          =   i
            # Stencil 
            iW          =   ii - 1
            iC          =   ii
            iE          =   ii + 1   
            # Boundaries 
            inW    =  i==1    ? false  : true
            DirW   = (i==1    && BC.type.W==:Dirichlet) ? 1. : 0.
            NeuW   = (i==1    && BC.type.W==:Neumann  ) ? 1. : 0.
            inE    =  i==nc ? false  : true
            DirE   = (i==nc && BC.type.E==:Dirichlet) ? 1. : 0.
            NeuE   = (i==nc && BC.type.E==:Neumann  ) ? 1. : 0.
            if inE
                K[ii,iE]    = ? 
            end
            K[ii,iC]        = ?
            if inW 
                K[ii,iW]    = ?
            end                            
            # Change right hand side due to boundary conditions ---
            rhs[i]  +=  ?
        end            
        T.T     .=   ?
    elseif alternative == 2            
        BackwardEuler1Dc!( ? )
    elseif alternative == 3
        for iter = 1:niter
            # Residual iteration
            ComputeResiduals1Dc!( ? )
            @printf("||R|| = %1.4e\n", norm(T.R)/length(T.R))            
            norm(T.R)/length(T.R) < ϵ ? break : nothing
            # Assemble linear system
            AssembleMatrix1Dc!( ? )
            # Solve for temperature correction: Cholesky factorisation
            Kc = cholesky(K.cscmatrix)
            # Solve for temperature correction: Back substitutions
            δT = ?              
            # Update temperature            
            T.T .= ?
        end        
        # Update temperature
        @. T.T0     =   ?
    end    
    # Calculate time ---
    Time    =   Time + Δt        
    # Plot Solution ---
    p = plot( ? )
    if save_fig == 1
        Plots.frame(anim)
    else
        display(p)
    end    
end

We now need to create and save the animation:

In [ ]:
# Speicher Animation ---------------------------------------------------- #
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)
else
    display(p)
end
foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))
# ----------------------------------------------------------------------- #